# TensorFlow: From Estimators to Keras
* Building a custom TensorFlow estimator (as a reference)
    1. Use **Census** classification dataset
    2. Create **feature columns** from the estimator
    3. Implement a **tf.data input_fn**
    4. Create a custom estimator using **tf.keras.layers**
    5. **Train** and **evaluate** the model
* Building a Functional Keras model and using tf.data APIs
    1. Modify the **input_fn** to process categorical features
    2. Build a Functional Keras Model
    3. Use the input_fn to fit the Keras model
    4. Configure **epochs** and **validation**
    5. Configure **callbacks** for **early stopping** and **checkpoints**
* Save and Load Keras model
* Export Keras model to saved_model
* Converting Keras model to estimator
* Concluding Remarks

In [1]:
import math
import os
import pandas as pd
import numpy as np
from datetime import datetime

import tensorflow as tf
from tensorflow import data

print "TensorFlow : {}".format(tf.__version__)

SEED = 19831060

TensorFlow : 1.12.0


## Download the Data

In [4]:
DATA_DIR='data'
# !mkdir $DATA_DIR
# !gsutil cp gs://cloud-samples-data/ml-engine/census/data/adult.data.csv $DATA_DIR
# !gsutil cp gs://cloud-samples-data/ml-engine/census/data/adult.test.csv $DATA_DIR
TRAIN_DATA_FILE = os.path.join(DATA_DIR, 'adult.data.csv')
EVAL_DATA_FILE = os.path.join(DATA_DIR, 'adult.test.csv')

In [5]:
TRAIN_DATA_SIZE = 32561
EVAL_DATA_SIZE = 16278

## Dataset Metadata

In [6]:
HEADER = ['age', 'workclass', 'fnlwgt', 'education', 'education_num',
               'marital_status', 'occupation', 'relationship', 'race', 'gender',
               'capital_gain', 'capital_loss', 'hours_per_week',
               'native_country', 'income_bracket']

HEADER_DEFAULTS = [[0], [''], [0], [''], [0], [''], [''], [''], [''], [''],
                       [0], [0], [0], [''], ['']]

NUMERIC_FEATURE_NAMES = ['age', 'education_num', 'capital_gain', 'capital_loss', 'hours_per_week']
CATEGORICAL_FEATURE_NAMES = ['gender', 'race', 'education', 'marital_status', 'relationship', 
                             'workclass', 'occupation', 'native_country']

FEATURE_NAMES = NUMERIC_FEATURE_NAMES + CATEGORICAL_FEATURE_NAMES
TARGET_NAME = 'income_bracket'
TARGET_LABELS = [' <=50K', ' >50K']
WEIGHT_COLUMN_NAME = 'fnlwgt'
NUM_CLASSES = len(TARGET_LABELS)

def get_categorical_features_vocabolary():
    data = pd.read_csv(TRAIN_DATA_FILE, names=HEADER)
    return {
        column: list(data[column].unique()) 
        for column in data.columns if column in CATEGORICAL_FEATURE_NAMES
    }

In [7]:
feature_vocabolary = get_categorical_features_vocabolary()
print(feature_vocabolary)

{'workclass': [' State-gov', ' Self-emp-not-inc', ' Private', ' Federal-gov', ' Local-gov', ' ?', ' Self-emp-inc', ' Without-pay', ' Never-worked'], 'relationship': [' Not-in-family', ' Husband', ' Wife', ' Own-child', ' Unmarried', ' Other-relative'], 'gender': [' Male', ' Female'], 'marital_status': [' Never-married', ' Married-civ-spouse', ' Divorced', ' Married-spouse-absent', ' Separated', ' Married-AF-spouse', ' Widowed'], 'race': [' White', ' Black', ' Asian-Pac-Islander', ' Amer-Indian-Eskimo', ' Other'], 'native_country': [' United-States', ' Cuba', ' Jamaica', ' India', ' ?', ' Mexico', ' South', ' Puerto-Rico', ' Honduras', ' England', ' Canada', ' Germany', ' Iran', ' Philippines', ' Italy', ' Poland', ' Columbia', ' Cambodia', ' Thailand', ' Ecuador', ' Laos', ' Taiwan', ' Haiti', ' Portugal', ' Dominican-Republic', ' El-Salvador', ' France', ' Guatemala', ' China', ' Japan', ' Yugoslavia', ' Peru', ' Outlying-US(Guam-USVI-etc)', ' Scotland', ' Trinadad&Tobago', ' Greece',

## Building a TensorFlow Custom Estimator

1. Creating feature columns
2. Creating model_fn
3. Create estimator using the model_fn
4. Define data input_fn
5. Define Train and evaluate experiment
6. Run experiment with parameters

### 1. Create feature columns

In [8]:
def create_feature_columns():
    
    feature_columns = []
    
    for column in NUMERIC_FEATURE_NAMES:
        feature_column = tf.feature_column.numeric_column(column)
        feature_columns.append(feature_column)
        
    for column in CATEGORICAL_FEATURE_NAMES:
        vocabolary = feature_vocabolary[column]
        embed_size = int(math.sqrt(len(vocabolary)))
        feature_column = tf.feature_column.embedding_column(
            tf.feature_column.categorical_column_with_vocabulary_list(column, vocabolary), 
            embed_size)
        feature_columns.append(feature_column)
        
    return feature_columns


### 2. Create model_fn
1. Use feature columns to create input_layer
2. Use tf.keras.layers to define the model architecutre and output
3. Use binary_classification_head for create EstimatorSpec

In [43]:
def model_fn(features, labels, mode, params):
    
    is_training = True if mode == tf.estimator.ModeKeys.TRAIN else False
    
    # model body
    def _inference(features, mode, params):
        
        feature_columns = create_feature_columns()
        input_layer = tf.feature_column.input_layer(features=features, feature_columns=feature_columns)
        dense_inputs = input_layer
        for i in range(len(params.hidden_units)):
            dense = tf.keras.layers.Dense(params.hidden_units[i], activation='relu')(dense_inputs)
            dense_dropout = tf.keras.layers.Dropout(params.dropout_prob)(dense, training=is_training)
            dense_inputs = dense_dropout
        fully_connected = dense_inputs  
        logits = tf.keras.layers.Dense(units=1, name='logits', activation=None)(fully_connected)
        return logits
    
    # model head
    head = tf.contrib.estimator.binary_classification_head(
        label_vocabulary=TARGET_LABELS,
        weight_column=WEIGHT_COLUMN_NAME
    )
    
    return head.create_estimator_spec(
        features=features,
        mode=mode,
        logits=_inference(features, mode, params),
        labels=labels,
        optimizer=tf.train.AdamOptimizer(params.learning_rate)
    )
    

### 3. Create estimator

In [44]:
def create_estimator(params, run_config):
    
    feature_columns = create_feature_columns()
    
    estimator = tf.estimator.Estimator(
        model_fn,
        params=params,
        config=run_config
    )
    
    return estimator

### 4. Data Input Function

In [45]:
def make_input_fn(file_pattern, batch_size, num_epochs, 
                  mode=tf.estimator.ModeKeys.EVAL):
    
    def _input_fn():
        dataset = tf.data.experimental.make_csv_dataset(
            file_pattern=file_pattern,
            batch_size=batch_size,
            column_names=HEADER,
            column_defaults=HEADER_DEFAULTS,
            label_name=TARGET_NAME,
            field_delim=',',
            use_quote_delim=True,
            header=False,
            num_epochs=num_epochs,
            shuffle= (mode==tf.estimator.ModeKeys.TRAIN)
        )
        
        iterator = dataset.make_one_shot_iterator()
        features, target = iterator.get_next()
        return features, target
    
    return _input_fn

### 5. Experiment Definition

In [46]:
def train_and_evaluate_experiment(params, run_config):
    
    # TrainSpec ####################################
    train_input_fn = make_input_fn(
        TRAIN_DATA_FILE,
        batch_size=params.batch_size,
        num_epochs=None,
        mode=tf.estimator.ModeKeys.TRAIN
    )
    
    train_spec = tf.estimator.TrainSpec(
        input_fn = train_input_fn,
        max_steps=params.traning_steps
    )
    ###############################################    
    
    # EvalSpec ####################################
    eval_input_fn = make_input_fn(
        EVAL_DATA_FILE,
        num_epochs=1,
        batch_size=params.batch_size,
    )

    eval_spec = tf.estimator.EvalSpec(
        name=datetime.utcnow().strftime("%H%M%S"),
        input_fn = eval_input_fn,
        steps=None,
        start_delay_secs=0,
        throttle_secs=params.eval_throttle_secs
    )
    ###############################################

    tf.logging.set_verbosity(tf.logging.INFO)
    
    if tf.gfile.Exists(run_config.model_dir):
        print("Removing previous artefacts...")
        tf.gfile.DeleteRecursively(run_config.model_dir)
            
    print ''
    estimator = create_estimator(params, run_config)
    print ''
    
    time_start = datetime.utcnow() 
    print("Experiment started at {}".format(time_start.strftime("%H:%M:%S")))
    print(".......................................") 

    tf.estimator.train_and_evaluate(
        estimator=estimator,
        train_spec=train_spec, 
        eval_spec=eval_spec
    )

    time_end = datetime.utcnow() 
    print(".......................................")
    print("Experiment finished at {}".format(time_end.strftime("%H:%M:%S")))
    print("")
    time_elapsed = time_end - time_start
    print("Experiment elapsed time: {} seconds".format(time_elapsed.total_seconds()))
    
    return estimator


### 6. Run Experiment with Parameters

In [47]:
MODELS_LOCATION = 'models/census'
MODEL_NAME = 'dnn_classifier'
model_dir = os.path.join(MODELS_LOCATION, MODEL_NAME)

params  = tf.contrib.training.HParams(
    batch_size=200,
    traning_steps=1000,
    hidden_units=[100, 70, 50],
    learning_rate=0.01,
    dropout_prob=0.2,
    eval_throttle_secs=0,
)

strategy = tf.contrib.distribute.MirroredStrategy(num_gpus=4)

run_config = tf.estimator.RunConfig(
    tf_random_seed=SEED,
    save_checkpoints_steps=200,
    keep_checkpoint_max=3,
    model_dir=model_dir,
    #train_distribute=strategy # use for multiple GPUs training
)

train_and_evaluate_experiment(params, run_config)


INFO:tensorflow:Using config: {'_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 3, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x121253e10>, '_model_dir': 'models/census/dnn_classifier', '_protocol': None, '_save_checkpoints_steps': 200, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': 19831060, '_save_summary_steps': 100, '_device_fn': None, '_experimental_distribute': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_evaluation_master': '', '_eval_distribute': None, '_global_id_in_cluster': 0, '_master': ''}

Experiment started at 17:51:39
.......................................
INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evalua

INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1000: models/census/dnn_classifier/model.ckpt-1000
INFO:tensorflow:Loss for final step: 73616.766.
.......................................
Experiment finished at 17:52:15

Experiment elapsed time: 35.751717 seconds


## Building a Keras Model
1. Implement a data input_fn process the data for the Keras model
2. Create the Keras model
3. Create the callbacks
4. Run the experiment

### 1. Data input_fn
1. Create lookups for categorical features vocabolary to numerical index
2. Process the dataset features to:
    * extrat the instance weight column
    * convert the categorical features to numerical index

In [285]:
def make_keras_input_fn(file_pattern, batch_size, mode=tf.estimator.ModeKeys.EVAL):
    
    mapping_tables = {}
        
    mapping_tables[TARGET_NAME] = tf.contrib.lookup.index_table_from_tensor(
        mapping=tf.constant(TARGET_LABELS))

    for feature_name in CATEGORICAL_FEATURE_NAMES:
        mapping_tables[feature_name] = tf.contrib.lookup.index_table_from_tensor(
            mapping=tf.constant(feature_vocabolary[feature_name]))
    try:
        tf.tables_initializer().run(session=tf.keras.backend.get_session()) 
    except:
        pass
            
    def _process_features(features, target):
        
        weight = features.pop(WEIGHT_COLUMN_NAME)
        target = mapping_tables[TARGET_NAME].lookup(target)
        for feature in CATEGORICAL_FEATURE_NAMES:
            features[feature] = mapping_tables[feature].lookup(features[feature])
        return features, target, weight
                        
    def _input_fn():
        
        dataset = tf.data.experimental.make_csv_dataset(
            file_pattern=file_pattern,
            batch_size=batch_size,
            column_names=HEADER,
            column_defaults=HEADER_DEFAULTS,
            label_name=TARGET_NAME,
            field_delim=',',
            use_quote_delim=True,
            header=False,
            shuffle= (mode==tf.estimator.ModeKeys.TRAIN)
        ).map(_process_features)

        return dataset
    
    return _input_fn

### 2. Create the keras model
1. Create the model architecture:
    * One input for each feature
    * Embedding layer for each categorical feature
    * Sigmoid output
2. Compile the model

In [49]:
def create_model(params):
    
    inputs = []
    to_concat = []
    
#     mapping_tables = {}
#     for feature_name in CATEGORICAL_FEATURE_NAMES:
#         mapping_tables[feature_name] = tf.contrib.lookup.index_table_from_tensor(
#             mapping=tf.constant(feature_vocabolary[feature_name]))
#     try:
#         tf.tables_initializer().run(session=tf.keras.backend.get_session()) 
#     except:
#         pass
    
    for column in HEADER:
        if column not in [WEIGHT_COLUMN_NAME, TARGET_NAME]:
            if column in NUMERIC_FEATURE_NAMES:
                numeric_input = tf.keras.layers.Input(shape=(1, ), name=column, dtype='float32')
                inputs.append(numeric_input)
                to_concat.append(numeric_input)
            else:
                categorical_input = tf.keras.layers.Input(shape=(1, ), name=column, dtype='int32')
                inputs.append(categorical_input)
                
#                 categorical_input_index = tf.keras.layers.Lambda(
#                     lambda string: mapping_tables[column].lookup(string))(categorical_input)

                vocabulary_size = len(feature_vocabolary[column])
                embed_size = int(math.sqrt(vocabulary_size))
                embedding = tf.keras.layers.Embedding(input_dim=vocabulary_size, 
                                                      output_dim=embed_size)(categorical_input)
                reshape = tf.keras.layers.Reshape(target_shape=(embed_size, ))(embedding)
                to_concat.append(reshape)
                    
    input_layer = tf.keras.layers.Concatenate(-1)(to_concat)    
    dense_inputs = input_layer
    for i in range(len(params.hidden_units)):
        dense = tf.keras.layers.Dense(params.hidden_units[i], activation='relu')(dense_inputs)
        dense_dropout = tf.keras.layers.Dropout(params.dropout_prob)(dense)#, training=is_training)
        dense_inputs = dense_dropout
    fully_connected = dense_inputs  
    logits = tf.keras.layers.Dense(units=1, name='logits', activation=None)(fully_connected)
    
    sigmoid = tf.keras.layers.Activation(activation='sigmoid', name='probability')(logits)

    # keras model
    model = tf.keras.models.Model(inputs=inputs, outputs=sigmoid)
    
    model.compile(
        loss='binary_crossentropy', 
        optimizer='adam', 
        metrics=['accuracy']
    )

    return model

In [313]:
!mkdir $model_dir/checkpoints
!ls $model_dir

mkdir: models/census/dnn_classifier/checkpoints: File exists
checkpoint                          model.ckpt-0.index
checkpoints                         model.ckpt-0.meta
export                              model.ckpt-1000.data-00000-of-00002
graph.pbtxt                         model.ckpt-1000.data-00001-of-00002
keras_classifier.h5                 model.ckpt-1000.index
model.ckpt-0.data-00000-of-00002    model.ckpt-1000.meta
model.ckpt-0.data-00001-of-00002


### 3. Define callbacks 
1. Early stopping callback
2. Checkpoints callback

In [50]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2),
    tf.keras.callbacks.ModelCheckpoint(
        os.path.join(model_dir,'checkpoints', 'model-{epoch:02d}.h5'), 
        monitor='val_loss', 
        period=1)
]

In [53]:
from keras.utils.training_utils import multi_gpu_model
model = create_model(params)
# model = multi_gpu_model(model, gpus=4) # This is to train the model with multiple GPUs
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
workclass (InputLayer)          (None, 1)            0                                            
__________________________________________________________________________________________________
education (InputLayer)          (None, 1)            0                                            
__________________________________________________________________________________________________
marital_status (InputLayer)     (None, 1)            0                                            
__________________________________________________________________________________________________
occupation (InputLayer)         (None, 1)            0                                            
__________________________________________________________________________________________________
relationsh

### 4. Run experiment

In [316]:
train_data = make_keras_input_fn(
    TRAIN_DATA_FILE,
    batch_size=params.batch_size,
    mode=tf.estimator.ModeKeys.TRAIN
)()


valid_data = make_keras_input_fn(
    EVAL_DATA_FILE,
    batch_size=params.batch_size,
    mode=tf.estimator.ModeKeys.EVAL
)()

steps_per_epoch = int(math.ceil(TRAIN_DATA_SIZE/float(params.batch_size)))
model.fit(
    train_data, 
    epochs=5, 
    steps_per_epoch=steps_per_epoch,
    validation_data=valid_data,
    validation_steps=steps_per_epoch,
    callbacks=callbacks
)

Epoch 1/5
163/163 [==============================] - 33s 201ms/step - loss: 1.5543 - acc: 0.7222 - val_loss: 1.3147 - val_acc: 0.7827
Epoch 2/5
163/163 [==============================] - 2s 15ms/step - loss: 1.3254 - acc: 0.7739 - val_loss: 1.1907 - val_acc: 0.7828
Epoch 3/5
163/163 [==============================] - 2s 14ms/step - loss: 1.2045 - acc: 0.7826 - val_loss: 1.1444 - val_acc: 0.8144
Epoch 4/5
163/163 [==============================] - 2s 14ms/step - loss: 1.1789 - acc: 0.7995 - val_loss: 1.1281 - val_acc: 0.8184
Epoch 5/5
163/163 [==============================] - 2s 12ms/step - loss: 1.3902 - acc: 0.7898 - val_loss: 1.4400 - val_acc: 0.7989


In [317]:
!ls $model_dir/checkpoints

model.01-1.31.hdf5 model.03-1.14.hdf5 model.05-1.44.hdf5
model.02-1.19.hdf5 model.04-1.13.hdf5


## Save and Load Keras Model for Prediction

In [290]:
keras_model_dir = os.path.join(model_dir, 'keras_classifier.h5')
model.save(keras_model_dir)
print("Keras model saved to: {}".format(keras_model_dir))
model = tf.keras.models.load_model(keras_model_dir)
print("Keras model loaded.")

Keras model saved to: models/census/dnn_classifier/keras_classifier.h5
Keras model loaded.


In [291]:
predict_data = make_keras_input_fn(
        EVAL_DATA_FILE,
        batch_size=5,
        mode=tf.estimator.ModeKeys.EVAL
    )()

predictions = map(
    lambda probability: TARGET_LABELS[0] if probability <0.5 else TARGET_LABELS[1], 
    model.predict(predict_data, steps=1)
)

print(list(predictions))

[' <=50K', ' <=50K', ' >50K', ' <=50K', ' <=50K']


## Export Keras Model as saved_model for tf.Serving

In [292]:
os.environ['MODEL_DIR'] = model_dir
export_dir = os.path.join(model_dir, 'export')

from tensorflow.contrib.saved_model.python.saved_model import keras_saved_model
keras_saved_model.save_keras_model(model, export_dir)


Consider using a TensorFlow optimizer from `tf.train`.
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['serving_default']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: models/census/dnn_classifier/export/temp-1547755904/saved_model.pb


'models/census/dnn_classifier/export/1547755904'

In [124]:
%%bash

saved_models_base=${MODEL_DIR}/export/
saved_model_dir=${saved_models_base}$(ls ${saved_models_base} | tail -n 1)
echo ${saved_model_dir}
ls ${saved_model_dir}
saved_model_cli show --dir=${saved_model_dir} --all

models/census/dnn_classifier/export/1547745765
assets
saved_model.pb
variables

MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['age'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 1)
        name: age:0
    inputs['capital_gain'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 1)
        name: capital_gain:0
    inputs['capital_loss'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 1)
        name: capital_loss:0
    inputs['education'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 1)
        name: education:0
    inputs['education_num'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 1)
        name: education_num:0
    inputs['gender'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 1)
        name: gender:0
    inputs['hours_per_week'] tensor_info:
        dtype: DT_FLOAT
        shape: 

## Convert to Estimator for Distributed Training...

In [125]:
estimator = tf.keras.estimator.model_to_estimator(model)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x13ad74a90>, '_model_dir': '/var/folders/hp/gzm_7hs931v5kt53p6rywh5w00fqrl/T/tmpXJ65ph', '_protocol': None, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_device_fn': None, '_experimental_distribute': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_evaluation_master': '', '_eval_distribute': None, '_train_distribute': None, '_master': ''}
